In [ ]:


# path_inference_data = Path("/afs/crc.nd.edu/group/dmsquare/vol2/myu2/ComparisonSentences/experiments/pair_scoring/Q11424/data_2k_hn.csv")
# dataset_inference = pd.read_csv(path_inference_data)
# X_infer = dataset_inference.iloc[:, 1:-1]
# Y_infer = dataset_inference.iloc[:, -1]


In [2]:
from sklearn.metrics import r2_score, explained_variance_score, max_error, mean_absolute_error, mean_squared_error, mean_poisson_deviance
from sklearn.ensemble import RandomForestRegressor
# as classification / ranking
import math
from sklearn.metrics import accuracy_score, precision_score, recall_score, ndcg_score, f1_score
import numpy as np
import scipy.stats as stats
import sklearn
from sklearn.model_selection import train_test_split

from pathlib import Path
import pandas as pd

# train
path_data = Path("/Users/mengxiayu/Documents/Research/ComparisonSentences/experiments/pair_scoring/global/data.tsv")
dataset = pd.read_csv(path_data)
dataset.drop_duplicates()
dataset = dataset.sample(frac = 1).reset_index(drop=True) # shuffle

X = dataset.iloc[:, :-1]
Y = dataset.iloc[:, -1]

# split dataset method 1: avoid overlap entities
def split_dataset(X, Y, method):
    if method == "no_overlap":
        existing_entities = set()
        for x in X.iloc[:2000, 0]:
            e1, e2, p, v1, v2 = eval(x)
            existing_entities.add(e1)
            existing_entities.add(e2)
        indices = []
        for ind in X.index:
            pair = X['pair'][ind]
            e1, e2, p, v1, v2 = eval(pair)
            if e1 not in existing_entities and e2 not in existing_entities:
                indices.append(ind)
        X_train = X.iloc[indices, :]
        Y_train = Y.iloc[indices]
        X_dev = X.iloc[:2000, :]
        Y_dev = Y.iloc[:2000]
        return X_train, Y_train, X_dev, Y_dev
    elif method == "random":
        X_train, X_dev, Y_train, Y_dev = train_test_split(X, Y, test_size=0.05, random_state=0)
        return X_train, Y_train, X_dev, Y_dev
    else:
        print("splitting method errir")
        return None
X_train, Y_train, X_dev, Y_dev = split_dataset(X, Y, "no_overlap")

X_train = X_train.iloc[:, 1:]
X_dev = X_dev.iloc[:, 1:]
print(len(X_train), len(X_dev))

regressor = RandomForestRegressor(n_estimators=130, random_state=0)
regressor.fit(X_train, Y_train)
y_pred_train = regressor.predict(X_train)
print("train_r2", r2_score(Y_train, y_pred_train))
y_pred = regressor.predict(X_dev)

def print_evaluation(truths, preds):
    print("test_r2", r2_score(truths, preds))
    print("test_mae", mean_absolute_error(truths, preds))
    print("test_mse", mean_squared_error(truths, preds))
    print("ndcg", ndcg_score(np.array([truths]), [preds]))
    print("kendalltau", stats.kendalltau(truths, preds)[0])
    y_pred = [1 if x > math.log(1.5) else 0 for x in preds]
    y_label = [1 if x > math.log(1.5) else 0 for x in truths]
    print("acc",accuracy_score(y_label, y_pred))
    print("p",precision_score(y_label, y_pred))
    print("r", recall_score(y_label, y_pred))
    print("f1", f1_score(y_label, y_pred))

print_evaluation(Y_dev, y_pred)
print(f"Importance of {len(regressor.feature_importances_)} features:")
for k,v in zip(regressor.feature_names_in_, regressor.feature_importances_):
    print(k,v)


57821 2000
train_r2 0.9446612422236144
test_r2 0.4202670872043277
test_mae 0.31109432109380897
test_mse 0.3993648216628805
ndcg 0.8790434271040206
kendalltau 0.6136835528119015
acc 0.9075
p 0.9022265246853823
r 0.9173228346456693
f1 0.9097120546608102
Importance of 35 features:
degree_e1 0.028321274491580204
degree_e2 0.031751543154737626
indegree_e1 0.030286899831846962
indegree_e2 0.03049909096455405
avg_rel_coverage 0.01448762160953726
min_rel_coverage 0.012695590433014638
max_rel_coverage 0.010852177304887555
diff_rel_coverage 0.012407358662921985
num_common_prop 0.010823876537201127
common_prop_rate 0.024152302781180867
num_common_feature 0.008471039408618633
common_feat_rate 0.03189010377062825
avg_common_prop_freq 0.03404318094228013
max_common_prop_freq 0.0
min_common_prop_freq 0.019997887690570527
avg_common_value_freq 0.021460435648831714
max_common_value_freq 0.008585177758027923
min_common_value_freq 0.0208074142582877
min_common_v_indegree 0.08674218245651077
max_common_v_

In [ ]:
path_data = Path("/Users/mengxiayu/Documents/Research/ComparisonSentences/experiments/pair_scoring/global/data.tsv")
dataset = pd.read_csv(path_data)


X = dataset.iloc[:, :-1]
Y = dataset.iloc[:, -1]
X.iloc[:, 0]

In [ ]:
# test
path_test_data = Path("/Users/mengxiayu/Documents/Research/ComparisonSentences/experiments/pair_scoring/Q5/test_data.tsv")
test_dataset = pd.read_csv(path_test_data)
X_test = test_dataset.iloc[:, 1:-1]
Y_test = test_dataset.iloc[:, -1]

y_pred_test = regressor.predict(X_test)
print_evaluation(Y_test, y_pred_test)


In [ ]:
for t in y_pred_test:
    print(t)

Grid Search

In [4]:
from sklearn.model_selection import GridSearchCV
param_search = {'n_estimators': range(80, 201, 20)}
gsearch = GridSearchCV(estimator=RandomForestRegressor(random_state=0),
                       param_grid=param_search,
                       scoring="r2", 
                       cv=3 )
gsearch.fit(X_train.iloc[:10000, :], Y_train.iloc[:10000])
gsearch.cv_results_, gsearch.best_params_, gsearch.best_score_

({'mean_fit_time': array([ 9.15480129, 11.13308446, 13.51055559, 15.36217539, 17.57106074,
         45.27998074, 22.03085303]),
  'std_fit_time': array([ 0.2602258 ,  0.23924122,  0.49525624,  0.49913398,  0.48800812,
         35.46893728,  1.09137444]),
  'mean_score_time': array([0.05588659, 0.08275557, 0.08194153, 0.1052746 , 0.10428667,
         0.13601661, 0.12406127]),
  'std_score_time': array([0.00084515, 0.02146036, 0.00506859, 0.01847758, 0.00123396,
         0.03098605, 0.00109461]),
  'param_n_estimators': masked_array(data=[80, 100, 120, 140, 160, 180, 200],
               mask=[False, False, False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'n_estimators': 80},
   {'n_estimators': 100},
   {'n_estimators': 120},
   {'n_estimators': 140},
   {'n_estimators': 160},
   {'n_estimators': 180},
   {'n_estimators': 200}],
  'split0_test_score': array([0.48673496, 0.4884008 , 0.48781553, 0.48837213, 0.48864157,
         0.4876

In [ ]:
param_test2 = {'max_depth':range(3,20,4), 'min_samples_split':range(10,201,30)}

gsearch = GridSearchCV(estimator=RandomForestRegressor(n_estimators=130, random_state=10),
                       param_grid=param_test2,
                       scoring="r2", 
                       cv=3 )
gsearch.fit(X_train, Y_train)
gsearch.cv_results_, gsearch.best_params_, gsearch.best_score_


In [ ]:
# test in another domain
regressor = RandomForestRegressor(n_estimators=130, random_state=0)
regressor.fit(X_train, Y_train)
y_pred_train = regressor.predict(X_train)
print("train_r2", r2_score(Y_train, y_pred_train))
Y_pred = regressor.predict(X_test)

# as classification
import math
from sklearn.metrics import accuracy_score, precision_score, recall_score, ndcg_score
print("ndcg", ndcg_score(np.array([Y_test]), [Y_pred]))
y_pred = [1 if x > math.log(1.5) else 0 for x in Y_pred]
y_label = [1 if x > math.log(1.5) else 0 for x in Y_test]
print("acc",accuracy_score(y_label, y_pred))
print("p",precision_score(y_label, y_pred))
print("r", recall_score(y_label, y_pred))

In [ ]:
precisions = []
recalls = []
for i in range(10):
    k = 0.1*i
    pred = [1 if x > math.log(1 + k) else 0 for x in y_pred]
    label = [1 if x > math.log(1.5) else 0 for x in Y_test]
    precisions.append(precision_score(label, pred))
    recalls.append(recall_score(label, pred))
print(precisions)
print(recalls)

In [ ]:
Y_pred

In [ ]:
X = dataset.iloc[:, 1:-1]
Y = dataset.iloc[:, -1]

train_score = []
test_score = []
n = 10
num_train_example = int(len(X_train) / n) 
for i in range(n):
    start = i*num_train_example
    end = (i+1)*num_train_example
    regressor = RandomForestRegressor(n_estimators=10, random_state=0)
    regressor.fit(X_train[:end], Y_train[:end])
    y_train_pred = regressor.predict(X_train[:end])
    train_score.append(r2_score(Y_train[:end], y_train_pred))

    y_test_pred = regressor.predict(X_test)
    test_score.append(r2_score(Y_test, y_test_pred))
    
import matplotlib.pyplot as plt
plt.plot([i for i in range(n)], train_score, label="train")
plt.plot([i for i in range(n)], test_score, label="test")
plt.legend()
plt.show()